# Measure Performance of Your Own Neural Network Model

This is continual series of assigment 

You can extend your own code from asssigment M3.1

#### Note. I changed the previous criteria for significant increase / decrease into 0.2 std from 1.0 std to avoid the safe answer

- Exercise1 Performance Evaluation for Trinary Classification Model
- Exercise2 Performance Evaluation for Return Prediction Model

# Exercise1  Performance Evaluation for Trinary Classification Model



### Data Download at: https://drive.google.com/open?id=1thjGhgnAm5k1zuSiWhGmlUJzBXM3IECi

This exercise is a little bit long exercise, that should give you an idea of a real world scenario. Feel free to look at the solution if you feel lost.

#### Requirements
1. Comparing Accuracy for both Train and Validation set data
  - Compare accurcacy according to the below materials
  - Visualize training history  
  - Check genelarization of your model
  - Refer
    - https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589
  - Visualization Hint
    - https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
2. Measure Model Accuracy for Classification Problem
  -  Now we are going to evaluate our model 
  - Accuracy, Recall, F1 Score based on Confusion Matrix
  - Refer definition of each scores
    - Confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix
    - Confusion matrix in Korean https://datascienceschool.net/view-notebook/731e0d2ef52c41c686ba53dcaf346f32/
  - Hint
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    
3. Measure Test Set Return based on the Simplest Strategy
  - Condition
    - Initial budget = 100
  - Strategy
    - If we predict up, then buy or hold (if we already bought)
    - If we predict down or no change then sell (if we already bought) or do nothing 
  - Draw your return

#### Procedures
- Preprocessing
  1. Data Import and Create Balanced Panel
  2. Create Target Variable
  3. Train / Test Split
  4. Create Sequences

- Training / Predicting Model
  1. Model Build
  2. Model Train
  3. Prediction
  4. Evaluation <- this assignment implement it

## Preprocessing

### 1. Data Import and Create Balanced Panel

In [0]:
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
DATA_PATH = "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

In [0]:
!ls "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

In [0]:
# Unzip Data
# It will take more than 5 min
import zipfile
import io

zf = zipfile.ZipFile(DATA_PATH+"crypto_data.zip", "r")
zf.extractall(DATA_PATH)

In [0]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

In [0]:
import pandas as pd

main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider

for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = DATA_PATH+f'crypto_data/{ratio}.csv'  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

### 2. Create Target Variable

In [0]:
SIGNIFICANT_CRITERIA = 0.2   # 0.2 std criteria 

currency_targets = ["BTC"]

for currency_target in currency_targets:
    main_df[currency_target+'-USD-TARGET'] = main_df[currency_target+'-USD_close'].shift(-FUTURE_PERIOD_PREDICT )
    main_df[currency_target+'-USD-TARGET-RETURN'] = (main_df[currency_target+'-USD-TARGET'] 
                                                                - main_df[currency_target+'-USD_close'])/main_df[currency_target+'-USD_close']

In [0]:
import numpy as np

def classify_trinary(values):
    gp_std = np.std(values)

    target = []
    for value in values:
        if SIGNIFICANT_CRITERIA*gp_std < value: # significant increase
            target.append(2)
        elif -SIGNIFICANT_CRITERIA*gp_std > value:  # significant decrease
            target.append(0)  
        else:
            target.append(1) # No significant change
            
    return target

In [0]:
main_df.head()

In [0]:
for currency_target in currency_targets:
    print("SIGNIFICANT_CRITERIA:", SIGNIFICANT_CRITERIA)
    main_df[currency_target+'-TARGET'] = main_df[currency_target+'-USD-TARGET-RETURN'].transform(classify_trinary)
    main_df.drop(columns=[currency_target+'-USD-TARGET', currency_target+'-USD-TARGET-RETURN'], inplace=True)

In [0]:
main_df.head()

### 3. Train / Test Split

In [0]:
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

test_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%

### 4. Create Sequences

In [0]:
from sklearn import preprocessing  # pip install sklearn ... if you don't have it!
from collections import deque
import random
import numpy as np

def sequence_generator(main_df, SEQ_LEN, suffle=True,seed=101):
    
  sequential_data = []  # this is a list that will CONTAIN the sequences
  queue = deque(maxlen = SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

  for i in main_df.values:  # iterate over the values
      queue.append([n for n in i[:-1]])  # store all but the target
      if len(queue) == SEQ_LEN:  # make sure we have 60 sequences!
          sequential_data.append([np.array(queue), i[-1]])  # append those bad boys!

  if suffle == True:
      random.seed(seed)
      random.shuffle(sequential_data)  # shuffle for good measure.

  X = []
  y = []

  for seq, target in sequential_data:  # going over our new sequential data
      X.append(seq)  # X is the sequences
      y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

  return np.array(X), y  # return X and y...and make X a numpy array!

In [0]:
train_x, train_y = sequence_generator(main_df , SEQ_LEN, suffle=True, seed=101)
test_x, test_y = sequence_generator(test_main_df , SEQ_LEN, suffle=True, seed=101)

In [0]:
print(train_x.shape, len(train_y))
print(test_x.shape, len(test_y))

## Up/ No Significant Change / Down Prediction Model

### 1. Model Build

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Flatten

def ex1_models(input_dim, output_dim):

  # you can try your own model!

  L1 = 50  # 30
  L2 = 30  # 20
  L3 = 20  # 10
  L4 = 10  # 5
  L5 = 5

  model = Sequential()
  model.add(Dense(L1, input_shape=input_dim, activation='relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(L2, activation='relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(L3, activation='relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(L4, activation='relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(L5, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(output_dim, activation='softmax'))

  model.compile(optimizer=tf.train.AdamOptimizer(0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

    
  return model

In [0]:
model1 = ex1_models(train_x.shape[1:], 3)

### 2. Model Train


In [0]:
BATCH_SIZE = 64 
NUM_ITERATIONS = 10

hist1 = model1.fit(train_x, tf.keras.utils.to_categorical(train_y, num_classes=None), 
              validation_split=0.2,
              batch_size = BATCH_SIZE,
              epochs = NUM_ITERATIONS)

### 3. Prediction

In [0]:
predictions = model1.predict_classes(test_x)

# Score model
score = model1.evaluate(test_x, tf.keras.utils.to_categorical(test_y, num_classes=None), 
                       verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

### 4. Evaluation

1. Comparing Accuracy for both Train and Validation set data
  - Compare accurcacy according to the below materials
  - Visualize training history
  - Check genelarization of your model
  - Refer
    - https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589
  - Visualization Hint
    - https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/    

In [0]:
# list all data in history
print(hist1.history.keys())

In [0]:
import matplotlib.pyplot as plt


2. Measure Model Accuracy for Classification Problem
  -  Now we are going to evaluate our model 
  - Accuracy, Recall, F1 Score based on Confusion Matrix
  - Refer definition of each scores
    - Confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix
    - Confusion matrix in Korean https://datascienceschool.net/view-notebook/731e0d2ef52c41c686ba53dcaf346f32/
  - Hint
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



3. Measure Test Set Return based on the Simplest Strategy
  - Condition
    - Initial budget = 100
  - Strategy
    - If we predict up, then buy or hold (if we already bought)
    - If we predict down or no change then sell (if we already bought) or do nothing 
  - Draw your return


# Exercise2 Performance Evaluation for Return Prediction Model

#### Requirements
1. Comparing loss for both Train and Validation set data
  - Compare loss according to the below materials
  - Visualize training history  
  - Check genelarization of your model
  - Refer
    - https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589
  - Visualization Hint
    - https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
2. Measure Model Accuracy for Continuous Value
  - Change Your Prediction Values into Up / Down Binary Variable
  - After then do the same things as in Exercise 1
    - Accuracy, Recall, F1 Score based on Confusion Matrix
    - Refer definition of each scores
      - Confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix
      - Confusion matrix in Korean https://datascienceschool.net/view-notebook/731e0d2ef52c41c686ba53dcaf346f32/
  - Hint
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    
3. Measure Test Set Return based on the Simplest Strategy
  - Condition
    - Initial budget = 100
  - Strategy
    - If we predict up, then buy or hold (if we already bought)
    - If we predict down, then sell (if we already bought) or do nothing 
  - Draw your return

#### Procedures
- Preprocessing
  1. Data Import and Create Balanced Panel
  2. Create Target Variable
  3. Train / Test Split
  4. Create Sequences
  
- Training / Predicting Model
  1. Model Build
  2. Model Train
  3. Prediction
  4. Evaluation

In [0]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

## Preprocessing

### 1. Data Import and Create Balanced Panel

In [0]:
import pandas as pd
from sklearn import preprocessing 

main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider

for ratio in ratios:  # begin iteration
  print(ratio)
  dataset = DATA_PATH+f'crypto_data/{ratio}.csv'  # get the full path to the file.
  df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

  # rename volume and close to include the ticker so we can still which close/volume is which:
  df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

  df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
  df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

  if len(main_df)==0:  # if the dataframe is empty
      main_df = df  # then it's just the current df
  else:  # otherwise, join this data to the main one
      main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

### 2. Create Target Variable

In [0]:
currency_targets = ["BTC"]

for currency_target in currency_targets:
  main_df[currency_target+'-USD-TARGET'] = main_df[currency_target+'-USD_close'].shift(-FUTURE_PERIOD_PREDICT )

  # scale up, you can do any other scaling methods!
  # how can we improve this part?
  # hint: reduce noise of future return
  main_df[currency_target+'-USD-TARGET-RETURN'] = (main_df[currency_target+'-USD-TARGET']-main_df[currency_target+'-USD_close'])/main_df[currency_target+'-USD_close']*100
  
  main_df.drop(columns=[currency_target+'-USD-TARGET'], inplace=True)

In [0]:
from sklearn import preprocessing  # pip install sklearn ... if you don't have it!

# Scaling Your Data
main_df.fillna(main_df.mean(), inplace=True)

In [0]:
main_df.head()

### 3. Train / Test Split

In [0]:
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

test_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%

### 4. Create Sequences

In [0]:
from sklearn import preprocessing  # pip install sklearn ... if you don't have it!
from collections import deque
import random
import numpy as np

def sequence_generator(main_df, SEQ_LEN, suffle=True,seed=101):
    
  sequential_data = []  # this is a list that will CONTAIN the sequences
  queue = deque(maxlen = SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

  for i in main_df.values:  # iterate over the values
      queue.append([n for n in i[:-1]])  # store all but the target
      if len(queue) == SEQ_LEN:  # make sure we have 60 sequences!
          sequential_data.append([np.array(queue), i[-1]])  # append those bad boys!

  if suffle == True:
      random.seed(seed)
      random.shuffle(sequential_data)  # shuffle for good measure.

  X = []
  y = []

  for seq, target in sequential_data:  # going over our new sequential data
      X.append(seq)  # X is the sequences
      y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

  return np.array(X), y  # return X and y...and make X a numpy array!

In [0]:
train_x, train_y = sequence_generator(main_df , SEQ_LEN, suffle=True, seed=101)
test_x, test_y = sequence_generator(test_main_df , SEQ_LEN, suffle=True, seed=101)

In [0]:
print(train_x.shape, len(train_y))
print(test_x.shape, len(test_y))

## Return Prediction Model

### 1. Model Build

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,\
CuDNNLSTM, BatchNormalization, Flatten, Activation

def ex2_models(input_dim, output_dim):

  # you can try your own model!
  
  L1 = 256  # 256
  L2 = 256  # 256
  L3 = 32  # 32

  model = Sequential()
  
  model.add(CuDNNLSTM(L1, input_shape=input_dim, return_sequences=True))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

  model.add(CuDNNLSTM(L2, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(BatchNormalization())

  model.add(CuDNNLSTM(L3))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())


  model.add(Flatten())
  model.add(Dense(output_dim))
  model.add(Activation('linear'))
  
  model.compile(optimizer=tf.train.AdamOptimizer(0.001),
            loss='mean_squared_error')
  
  return model

### 2. Model Train

In [0]:
model2 = ex2_models(train_x.shape[1:], 1)

In [0]:
BATCH_SIZE = 64 
NUM_ITERATIONS = 10

hist2 = model2.fit(train_x, train_y,
              validation_split=0.2,
              batch_size = BATCH_SIZE,
              epochs = NUM_ITERATIONS)

### 3. Prediction

In [0]:
predictions = model2.predict(test_x)

# Score model
score = model2.evaluate(test_x, test_y,
                       verbose=0)

print('Test loss:', score) # this is mean_squared_error 

### 4. Evaluation

1. Comparing Loss for both Train and Validation set data
  - Compare loss according to the below materials
  - Visualize training history  
  - Check genelarization of your model
  - Refer
    - https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589
  - Visualization Hint
    - https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/    

In [0]:
# list all data in history
print(hist2.history.keys())

In [0]:
import matplotlib.pyplot as plt


    
2. Measure Model Accuracy for Continuous Value
  - Change Your Prediction Values into Up / Down Binary Variable
  - After then do the same things as in Exercise 1
    - Accuracy, Recall, F1 Score based on Confusion Matrix
    - Refer definition of each scores
      - Confusion matrix https://en.wikipedia.org/wiki/Confusion_matrix
      - Confusion matrix in Korean https://datascienceschool.net/view-notebook/731e0d2ef52c41c686ba53dcaf346f32/
  - Hint
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


3. Measure Test Set Return based on the Simplest Strategy
  - Condition
    - Initial budget = 100
  - Strategy
    - If we predict up, then buy or hold (if we already bought)
    - If we predict down, then sell (if we already bought) or do nothing 
  - Draw your return
